In [1]:
import numpy as np
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [5]:
actions = ['good', 'five', 'fit','V','piece']

good_data = np.load('/content/drive/MyDrive/dataset_1/seq_good_1736319783.npy')
five_data = np.load('/content/drive/MyDrive/dataset_1/seq_five_1736319783.npy')
fit_data = np.load('/content/drive/MyDrive/dataset_1/seq_fit_1736319783.npy')
V_data = np.load('/content/drive/MyDrive/dataset_1/seq_V_1736319783.npy')
piece_data = np.load('/content/drive/MyDrive/dataset_1/seq_piece_1736319783.npy')

print("Good shape:", good_data.shape)
print("Five shape:", five_data.shape)
print("Fit shape:", fit_data.shape)
print("V shape:", V_data.shape)
print("Piece shape:", piece_data.shape)

# Concatenate data
data = np.concatenate([good_data, five_data, fit_data, V_data, piece_data], axis=0)
print("Concatenated data shape:", data.shape)

Good shape: (527, 30, 100)
Five shape: (351, 30, 100)
Fit shape: (329, 30, 100)
V shape: (402, 30, 100)
Piece shape: (398, 30, 100)
Concatenated data shape: (2007, 30, 100)


In [6]:
x_data = data[:, :, :-1]
labels = data[:, 0, -1]

print(x_data.shape)
print(labels.shape)

(2007, 30, 99)
(2007,)


In [7]:
from tensorflow.keras.utils import to_categorical

y_data = to_categorical(labels, num_classes=len(actions))
y_data.shape

(2007, 5)

In [8]:
from sklearn.model_selection import train_test_split

x_data = x_data.astype(np.float32)
y_data = y_data.astype(np.float32)

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=2021)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(1806, 30, 99) (1806, 5)
(201, 30, 99) (201, 5)


In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, LeakyReLU

model = Sequential([
    GRU(64, input_shape=x_train.shape[1:3], return_sequences=False),  # GRU 레이어
    LeakyReLU(alpha=0.3),  # LeakyReLU 적용
    Dropout(0.5),  # 50% 드롭아웃
    Dense(32),
    LeakyReLU(alpha=0.3),  # LeakyReLU 적용
    Dropout(0.3),  # 30% 드롭아웃
    Dense(len(actions), activation='softmax')  # 출력층
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_1 (GRU)                          │ (None, 64)                  │          31,680 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_2 (LeakyReLU)            │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_3 (LeakyReLU)            │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 5)                   │             165 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 33,925 (132.52 KB)

 Trainable params: 33,925 (132.52 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

history = model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=20,
    callbacks=[
        ModelCheckpoint('models/model.keras', monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=50, verbose=1, mode='auto')
    ]
)

Epoch 1/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - acc: 0.4992 - loss: 1.2472
Epoch 1: val_acc improved from -inf to 0.96020, saving model to models/model.keras
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - acc: 0.5018 - loss: 1.2422 - val_acc: 0.9602 - val_loss: 0.3809 - learning_rate: 0.0010
Epoch 2/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - acc: 0.8992 - loss: 0.4364
Epoch 2: val_acc improved from 0.96020 to 0.99005, saving model to models/model.keras
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - acc: 0.8994 - loss: 0.4354 - val_acc: 0.9900 - val_loss: 0.1271 - learning_rate: 0.0010
Epoch 3/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - acc: 0.9469 - loss: 0.2217
Epoch 3: val_acc improved from 0.99005 to 0.99502, saving model to models/model.keras
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - acc: 0.9470 - loss: 0.2214 - val_acc: 0.9950 - val_loss: 0.0562 - learning_rate: 0.0010
Epoch 4/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - acc: 0.9559 - loss: 0.1622
Epoch 4: val_acc improved from 0.99

In [30]:
from sklearn.metrics import multilabel_confusion_matrix
from tensorflow.keras.models import load_model

model = load_model('models/model.keras')

y_pred = model.predict(x_val)

multilabel_confusion_matrix(np.argmax(y_val, axis=1), np.argmax(y_pred, axis=1))

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


array([[[151,   0],
        [  0,  50]],

       [[169,   0],
        [  0,  32]],

       [[167,   0],
        [  0,  34]],

       [[152,   0],
        [  0,  49]],

       [[165,   0],
        [  0,  36]]])

In [23]:
# 모델 저장
model.save('model.h5')